In [2]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
dataset_path = 'fer2013.csv'
image_size=(48,48)
 
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

C:\Users\HP\Anaconda3\envs\deepl\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [8]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np
 
# parameters
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = 'models/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 
# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
 
# callbacks
base_path=r'fer2013'
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 46, 46, 8)    72          input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 46, 46, 8)    32          conv2d_29[0][0]                  
__________________________________________________________________________________________________
activation_25 (Activation)      (None, 46, 46, 8)    0           batch_normalization_57[0][0]     
__________________________________________________________________________________________________
conv2d_30 

Epoch 1/110
898/897 [==============================] - 139s 155ms/step - loss: 1.7366 - acc: 0.3475 - val_loss: 1.5963 - val_acc: 0.4154

Epoch 00001: val_loss improved from inf to 1.59631, saving model to fer2013_mini_XCEPTION.01-0.42.hdf5
Epoch 2/110
898/897 [==============================] - 104s 116ms/step - loss: 1.4893 - acc: 0.4415 - val_loss: 1.5484 - val_acc: 0.4468

Epoch 00002: val_loss improved from 1.59631 to 1.54842, saving model to fer2013_mini_XCEPTION.02-0.45.hdf5
Epoch 3/110
898/897 [==============================] - 104s 116ms/step - loss: 1.3846 - acc: 0.4836 - val_loss: 1.3206 - val_acc: 0.5143

Epoch 00003: val_loss improved from 1.54842 to 1.32063, saving model to fer2013_mini_XCEPTION.03-0.51.hdf5
Epoch 4/110
898/897 [==============================] - 104s 116ms/step - loss: 1.3195 - acc: 0.5092 - val_loss: 1.4429 - val_acc: 0.4772

Epoch 00004: val_loss did not improve from 1.32063
Epoch 5/110
898/897 [==============================] - 104s 116ms/step - loss: 1

898/897 [==============================] - 103s 115ms/step - loss: 0.9849 - acc: 0.6327 - val_loss: 1.0643 - val_acc: 0.6169

Epoch 00040: val_loss did not improve from 1.02018
Epoch 41/110
898/897 [==============================] - 103s 115ms/step - loss: 0.9754 - acc: 0.6356 - val_loss: 1.1272 - val_acc: 0.5911

Epoch 00041: val_loss did not improve from 1.02018
Epoch 42/110
898/897 [==============================] - 104s 116ms/step - loss: 0.9784 - acc: 0.6321 - val_loss: 1.0607 - val_acc: 0.6131

Epoch 00042: val_loss did not improve from 1.02018
Epoch 43/110
898/897 [==============================] - 105s 116ms/step - loss: 0.9683 - acc: 0.6365 - val_loss: 1.0245 - val_acc: 0.6236

Epoch 00043: val_loss did not improve from 1.02018
Epoch 44/110
898/897 [==============================] - 105s 116ms/step - loss: 0.9658 - acc: 0.6415 - val_loss: 1.0402 - val_acc: 0.6181

Epoch 00044: val_loss did not improve from 1.02018
Epoch 45/110
898/897 [==============================] - 111s 12


Epoch 00080: val_loss did not improve from 0.96298
Epoch 81/110
898/897 [==============================] - 105s 117ms/step - loss: 0.8335 - acc: 0.6904 - val_loss: 0.9681 - val_acc: 0.6499

Epoch 00081: val_loss did not improve from 0.96298
Epoch 82/110
898/897 [==============================] - 105s 117ms/step - loss: 0.8338 - acc: 0.6917 - val_loss: 0.9653 - val_acc: 0.6520

Epoch 00082: val_loss did not improve from 0.96298
Epoch 83/110
898/897 [==============================] - 106s 118ms/step - loss: 0.8361 - acc: 0.6892 - val_loss: 0.9662 - val_acc: 0.6530

Epoch 00083: val_loss did not improve from 0.96298
Epoch 84/110
898/897 [==============================] - 105s 117ms/step - loss: 0.8338 - acc: 0.6887 - val_loss: 0.9656 - val_acc: 0.6532

Epoch 00084: val_loss did not improve from 0.96298
Epoch 85/110
898/897 [==============================] - 105s 117ms/step - loss: 0.8293 - acc: 0.6918 - val_loss: 0.9676 - val_acc: 0.6530

Epoch 00085: val_loss did not improve from 0.9629